In [1]:
import sys

sys.path.insert(1, '/home/peps/Documents/tesis_codigo')

In [2]:
import helpers.preprocesamiento as procs
import spacy
import pickle

In [3]:
with open("pickles/lectura.pickle", "rb") as f:
    df, f_ini, f_fin = pickle.load(f) 

In [4]:
df_char_r = procs.remove_chars(df)

In [5]:
df_ = df_char_r
nlp = spacy.load('es_core_news_md')

In [6]:
stop_words = ["y", "a", "ante", "a", "bajo", "con", "de", "desde", "durante", "en", "entre", "excepto", "hacia",
              "hasta", "mediante", "para", "por", "salvo", "según", "sin", "sobre", "tras", "y", "e", "ni", "o", "u",
              "que", "si", "como", "donde", "quien", "cual", "cuyo", "cuanto", "el", "lalos", "las"]

for word in stop_words:
    nlp.vocab[word].is_stop = True

In [7]:
docs = list(df_['text_clean'])

In [8]:
docs = procs.entidades(docs, nlp)

In [9]:
docs = procs.bigramas(docs)

In [10]:
dictionary = procs.filtrar_extremos(docs)

In [11]:
# %%
corpus = [dictionary.doc2bow(doc) for doc in docs]

In [12]:
# BORRAR TWEETS VACIOS
id_borrar = [i for i in range(0, len(corpus)) if len(corpus[i]) == 0]
df_tweets_vacios = df.drop(index = id_borrar)
df = df.reset_index(drop=True)
docs = list(df['text'])
docs = procs.entidades(docs, nlp)
docs = procs.bigramas(docs)
corpus = [dictionary.doc2bow(doc) for doc in docs]

In [13]:
# AUTHOR2DOC
author2doc = {}
for aut in df.screen_name.unique():
    author2doc[aut] = []

for index, row in df.iterrows():
    author2doc[row['screen_name']].append(index)

In [14]:
with open("pickles/preprocesamiento.pickle", "wb") as f:
    pickle.dump((corpus, dictionary, author2doc) , f)